## Projet RI

### Prédiction des caractéristiques physiques du carbone noir à l'aide de variables Lidar

Ci-dessous est codée une fonction qui permet de renseigner les caractéristiques physiques d'une particule de carbone noir. Ces caractéristiques sont obtenues à partir de 4 élèments : la valeur de son Color Ration associé, de son Backscattering Angle Effect, de sa dimension fractale et pourcentage de coating.
Pour obtenir ces informations, différents modèles de Machine Learning ont été entrainés au préalable et seulement les modèles donnant les meilleurs résultats ont été sélectionnés. Dans notre cas les meilleurs résultats ont été obtenus :
- pour 'primary_particle_size (nm)': avec 'GBR',
- pour 'vol_equi_radius_outer (nm)': avec 'KRR',
- pour 'vol_equi_radius_inner (nm)': avec 'KRR',
- pour 'equi_mobility_dia (nm)': avec 'KRR',
- pour 'mass_bc (g)': avec 'KRR'

Pour utiliser la fonction 'Predict_parameters', veuillez renseigner les éléments suivants : 
- 'cr' : Color Ratio (float)
- 'bae' : Backscattering Angle Effect (float)
- 'fractal_dimension' : Dimension Fractale (float)
- 'fraction_of_coating' : Pourcentage de Coating (integer, %)

In [1]:
import numpy as np
from joblib import load
import os
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [5]:
# Si vous n'avez pas les valeurs de CR ou BAE rentrez 0 comme argument 
# Conseil : pour les volumes la prédiction sera plus précise en utilisant 
def Predict_parameters(cr, bae, fractal_dimension, fraction_of_coating):

    # Variable à prédire
    var_target = {
         'primary_particle_size (nm)': 'GBR',
         'vol_equi_radius_outer (nm)': 'KRR',
         'vol_equi_radius_inner (nm)': 'KRR',
         'equi_mobility_dia (nm)': 'KRR',
         'mass_bc (g)': 'KRR'}
     # Répertoire des modèles sauvegardés
    if cr == 0 : 
        model_dir = "./Best_models/L_X_BAE_best_models"
        input_data = np.array([bae, fractal_dimension, fraction_of_coating]).reshape(1, -1)
        input_data =  scaler.fit_transform(input_data)
        r2_values = {
            'R2 primary_particle_size (nm)': 1 ,
            'R2 vol_equi_radius_outer (nm)': 0.917,
            'R2 vol_equi_radius_inner (nm)': 0.925,
            'R2 equi_mobility_dia (nm)': 0.915,
            'R2 mass_bc (g)': 0.85
        }
    elif bae == 0 :
        model_dir = "./Best_models/L_X_CR_best_models"
        input_data = np.array([cr, fractal_dimension, fraction_of_coating]).reshape(1, -1)
        input_data =  scaler.fit_transform(input_data)
        r2_values = {
            'R2 primary_particle_size (nm)': 1 ,
            'R2 vol_equi_radius_outer (nm)': 0.916,
            'R2 vol_equi_radius_inner (nm)': 0.924,
            'R2 equi_mobility_dia (nm)': 0.912,
            'R2 mass_bc (g)': 0.53
        }
        
    else : 
        model_dir = "./Best_models/L_X_best_models"
        input_data = np.array([cr, bae, fractal_dimension, fraction_of_coating]).reshape(1, -1)
        input_data =  scaler.fit_transform(input_data) 
        r2_values = {
            'R2 primary_particle_size (nm)': 1,
            'R2 vol_equi_radius_outer (nm)': 0.916,
            'R2 vol_equi_radius_inner (nm)': 0.924,
            'R2 equi_mobility_dia (nm)': 0.912,
            'R2 mass_bc (g)': 0.54
        }
        
        

    # Données d'entrée pour la prédiction

    predictions = {}
    
    #Pour chaque variable target on effectue la prédiction sur son modèle associé
    for target, model_type in var_target.items():
        
        # Chemin du modèle
        model_path = os.path.join(model_dir, f'{target}_best_model_{model_type}.joblib')
        # Vérification de l'existence du fichier
        if not os.path.exists(model_path):
            raise FileNotFoundError(f"Modèle pour '{target}' introuvable à {model_path}")

        # Chargement du modèle
        model = load(model_path)
        # Prédiction
        predictions[target] = np.abs(model.predict(input_data)[0]) #ces valeurs doivent être positives 

    return predictions,r2_values


### Exemple d'utilisation : 

#### Exemple fictif 

In [6]:
# Si pas de valeurs de CR et BAE mettre 0 à la place 
# Les valeurs de R2 nous donne une indication sur l'accuracy 

#'cr'=0.6 
#'bae'=-3.9
#'fractal_dimension'=1.5
#'fraction_of_coating'=0%

Predict_parameters(0, -3.9 , 1.5, 0)

ModuleNotFoundError: No module named '_loss'

#### Exemple avec valeurs du jeu de données 

In [64]:
cr=0.66628753296408
bae= -3.20854360536531
fractal_dimension=1.7
fraction_of_coating =40 #%

Predict_parameters(cr, bae ,fractal_dimension , fraction_of_coating)

# On attent 
# primary_particle_size (nm) = 17,8 
#'vol_equi_radius_outer (nm)' = 165
# vol_equi_radius_inner (nm) = 139 
# equi_mobility_dia (nm) = 855
# mass_bc (g)' = 1,7 e-14

({'primary_particle_size (nm)': 16.899999998433167,
  'vol_equi_radius_outer (nm)': 93.7815555907205,
  'vol_equi_radius_inner (nm)': 82.80940847534984,
  'equi_mobility_dia (nm)': 372.22824042147596,
  'mass_bc (g)': 4.065502164008171e-15},
 {'R2 primary_particle_size (nm)': 1,
  'R2 vol_equi_radius_outer (nm)': 0.916,
  'R2 vol_equi_radius_inner (nm)': 0.924,
  'R2 equi_mobility_dia (nm)': 0.912,
  'R2 mass_bc (g)': 0.54})

In [ ]:
cr=0.697442625
bae= -2,847423437
fractal_dimension=1.5
fraction_of_coating =15 #%

Predict_parameters(cr, bae ,fractal_dimension , fraction_of_coating)

# On attent 
# primary_particle_size (nm) = 15,8
#'vol_equi_radius_outer (nm)' = 116,4153954
# vol_equi_radius_inner (nm) = 110,520945 
# equi_mobility_dia (nm) = 533,007323
# mass_bc (g)' = 8,48E-15

In [ ]:
cr=0,606022585
bae= -3,957699995
fractal_dimension=1.7
fraction_of_coating =0 #%

Predict_parameters(cr, bae ,fractal_dimension , fraction_of_coating)

# On attent 
# primary_particle_size (nm) = 16,2
#'vol_equi_radius_outer (nm)' = 12,85794852
# vol_equi_radius_inner (nm) = 11,90550789
# equi_mobility_dia (nm) = 18,07237419
# mass_bc (g)' =1,06E-17

In [ ]:
cr=0,582127917
bae= -4,275580337
fractal_dimension=1,9
fraction_of_coating =30 #%

Predict_parameters(cr, bae ,fractal_dimension , fraction_of_coating)

# On attent 
# primary_particle_size (nm) =16,9
#'vol_equi_radius_outer (nm)' = 35,15341661
# vol_equi_radius_inner (nm) = 31,20125735
# equi_mobility_dia (nm) = 82,33336731
# mass_bc (g)' = 1,91E-16